# Data Processing
Author: Huiting Song

In [40]:
# Import packages
import pandas as pd
import sklearn
import numpy as np
import nltk
import json
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import KNNImputer

In [42]:
from codecarbon import EmissionsTracker
experiment_name = "dataprocessing-POWER"

tracker = EmissionsTracker(
    output_dir="/Users/crystal/Desktop/ANLY5550/codecarbon",
    output_file=f"{experiment_name}_emissions.csv",
    log_level="error",  # comment out this line to see regular output
)
tracker.start()

## **Dataset from NASA POWER**
This is satellite data from NASA. The data includes the solar and wind geometry variables, atmospheric and tamporal conditions variables in three areas -- Arlington, Shenandoah, Shenandoah -- in Virgina from 2021.01.01 to 2024.03.31. The processing flow for the datasets are similar, including data merging, variables selection, and NA check. 

The selected variables will be:<p>
- Solar Geometry Variables:<p><p>
`YEAR`, `MO`, `DY`: These time variables can capture seasonal and daily variations in solar irradiation.<p>
`ALLSKY_SFC_SW_DWN`, `CLRSKY_SFC_SW_DWN`: All-sky and clear-sky solar irradiance are direct measures of solar power potential.<p>
- Atmospheric and Temporal Variables:<p><p>
`ALLSKY_KT`: The clearness index, which indicates the fraction of sunlight not obstructed by clouds.<p>
`T2M`, `T2MDEW`, `T2MWET`: Air temperature, dew point, and wet bulb temperature can affect atmospheric clarity.<p>
`PS`: Surface pressure influences atmospheric depth, affecting sunlight transmission and wind patterns and speed.<p>
`T2M`, `T2M_MAX`, `T2M_MIN`, `T2M_RANGE`: Temperature affects air density, which influences wind turbine efficiency.<p>
- Wind Variables:<p><p>
`WS10M`, `WS10M_MAX`, `WS10M_MIN`, `WS10M_RANGE`: Wind speed at 10 meters including maximum, minimum, and range.<p>
`WD10M`: Wind direction at 10 meters.<p>
`WS50M`, `WS50M_MAX`, `WS50M_MIN`, `WS50M_RANGE`: Wind speed at 50 meters; higher altitude winds are critical for utility-scale turbines.<p>
`WD50M`: Wind direction at 50 meters.


### Arlington

In [43]:
Arlington_SolarTemp = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Arlington_Daily_Temp_Solar.csv")
Arlington_SolarTemp.head()

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN
0,2021,1,1,0.47,2.49,0.12,299.76,8.98,46.16,1.35,0.02,0.09,1.07,-0.09,0.49,1.00,4.16,2.77,-1.39
1,2021,1,2,2.20,2.76,0.54,299.48,39.69,50.02,4.94,0.07,0.34,4.35,2.97,3.66,3.97,8.38,9.91,1.54
2,2021,1,3,0.47,2.65,0.11,310.62,8.85,48.22,1.32,0.02,0.09,1.99,1.27,1.63,1.83,2.79,3.30,0.51
3,2021,1,4,1.31,2.53,0.31,288.55,23.64,43.89,3.16,0.03,0.16,2.22,0.28,1.25,1.65,9.23,7.33,-1.90
4,2021,1,5,1.03,2.53,0.25,300.42,19.10,43.59,2.62,0.02,0.13,2.36,0.71,1.54,1.64,5.30,5.40,0.09


In [44]:
Arlington_Wind = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Arlington_Daily_Wind.csv")
Arlington_Wind.head()

,YEAR,MO,DY,PS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,WS2M
0,2021,1,1,101.79,3.56,5.23,1.53,3.71,89.69,5.63,7.25,2.84,4.41,91.75,0.47,2.49,2.40
1,2021,1,2,100.68,4.41,6.74,2.90,3.84,223.38,6.85,9.28,5.45,3.84,223.81,2.20,2.76,2.90
2,2021,1,3,100.69,3.97,4.94,2.70,2.24,175.56,5.96,8.43,3.60,4.83,176.69,0.47,2.65,2.72
3,2021,1,4,100.74,3.02,4.76,1.48,3.28,335.19,4.65,8.35,2.23,6.12,335.62,1.31,2.53,1.96
4,2021,1,5,100.56,2.10,4.09,1.13,2.95,329.69,3.49,7.49,1.26,6.23,329.69,1.03,2.53,1.29


In [45]:
# Merging DataFrames
Arlington = pd.merge(Arlington_SolarTemp, Arlington_Wind, on=['YEAR', 'MO', 'DY'])

In [46]:
# Showing all 33 variables
Arlington.columns

Index(['YEAR', 'MO', 'DY', 'ALLSKY_SFC_SW_DWN_x', 'CLRSKY_SFC_SW_DWN_x',
       'ALLSKY_KT', 'ALLSKY_SFC_LW_DWN', 'ALLSKY_SFC_PAR_TOT',
       'CLRSKY_SFC_PAR_TOT', 'ALLSKY_SFC_UVA', 'ALLSKY_SFC_UVB',
       'ALLSKY_SFC_UV_INDEX', 'T2M', 'T2MDEW', 'T2MWET', 'TS', 'T2M_RANGE',
       'T2M_MAX', 'T2M_MIN', 'PS', 'WS10M', 'WS10M_MAX', 'WS10M_MIN',
       'WS10M_RANGE', 'WD10M', 'WS50M', 'WS50M_MAX', 'WS50M_MIN',
       'WS50M_RANGE', 'WD50M', 'ALLSKY_SFC_SW_DWN_y', 'CLRSKY_SFC_SW_DWN_y',
       'WS2M'],
      dtype='object')

In [47]:
# Selecting the variables that are needed for modeling
columns_to_select = [
    'YEAR', 'MO', 'DY',
    'ALLSKY_SFC_SW_DWN_x', 'CLRSKY_SFC_SW_DWN_x',
    'ALLSKY_KT', 'T2M', 'T2MDEW', 'T2MWET', 'PS',
    'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WD10M',
    'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE', 'WD50M'
]

# Selecting the columns from the DataFrame
Arlington_df = Arlington[columns_to_select]

In [48]:
# Changing variables name
Arlington_df = Arlington_df.rename(columns={
    'ALLSKY_SFC_SW_DWN_x': 'ALLSKY_SFC_SW_DWN',
    'CLRSKY_SFC_SW_DWN_x': 'CLRSKY_SFC_SW_DWN'
})

# Creating a column named "Date" by transfering the time variables to date
Arlington_df['DATE'] = pd.to_datetime({'year': Arlington_df['YEAR'], 'month': Arlington_df['MO'], 'day': Arlington_df['DY']})

In [49]:
# Displaying the dataframe
pd.options.display.max_rows = 10 
display(Arlington_df)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.47,2.49,0.12,1.07,-0.09,0.49,101.79,...,5.23,1.53,3.71,89.69,5.63,7.25,2.84,4.41,91.75,2021-01-01
1,2021,1,2,2.20,2.76,0.54,4.35,2.97,3.66,100.68,...,6.74,2.90,3.84,223.38,6.85,9.28,5.45,3.84,223.81,2021-01-02
2,2021,1,3,0.47,2.65,0.11,1.99,1.27,1.63,100.69,...,4.94,2.70,2.24,175.56,5.96,8.43,3.60,4.83,176.69,2021-01-03
3,2021,1,4,1.31,2.53,0.31,2.22,0.28,1.25,100.74,...,4.76,1.48,3.28,335.19,4.65,8.35,2.23,6.12,335.62,2021-01-04
4,2021,1,5,1.03,2.53,0.25,2.36,0.71,1.54,100.56,...,4.09,1.13,2.95,329.69,3.49,7.49,1.26,6.23,329.69,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,1.03,-999.00,-999.00,7.55,6.65,7.10,100.97,...,3.89,1.27,2.62,131.56,3.55,5.34,2.41,2.93,132.56,2024-03-27
1182,2024,3,28,1.24,-999.00,-999.00,6.85,4.46,5.65,100.68,...,8.65,0.99,7.66,304.88,7.48,10.83,1.63,9.19,305.06,2024-03-28
1183,2024,3,29,5.88,-999.00,-999.00,7.04,2.35,4.69,100.47,...,9.83,3.72,6.11,308.12,9.54,12.87,5.89,6.98,308.50,2024-03-29
1184,2024,3,30,3.18,-999.00,-999.00,9.04,5.31,7.17,100.30,...,5.30,2.59,2.71,250.94,6.10,8.95,3.47,5.47,252.19,2024-03-30


In [50]:
# checking the data types
print(Arlington_df.dtypes) # nothing needs change

YEAR                          int64
MO                            int64
DY                            int64
ALLSKY_SFC_SW_DWN           float64
CLRSKY_SFC_SW_DWN           float64
                          ...      
WS50M_MAX                   float64
WS50M_MIN                   float64
WS50M_RANGE                 float64
WD50M                       float64
DATE                 datetime64[ns]
Length: 21, dtype: object


In [51]:
# checking the NA value
print(Arlington_df.isna().sum()) # no missing value in each columns

YEAR                 0
MO                   0
DY                   0
ALLSKY_SFC_SW_DWN    0
CLRSKY_SFC_SW_DWN    0
                    ..
WS50M_MAX            0
WS50M_MIN            0
WS50M_RANGE          0
WD50M                0
DATE                 0
Length: 21, dtype: int64


**Note:** Even though no missing value has been found in the dataset, we can observe that there are some abnormal value "-999.00" in some columns. In scientific and meteorological datasets like the NASA POWER data, the value -999.00 often represents a placeholder for missing or undefined data. For handling this, we need to look at the portion of undefined data first and then decide the imputation method to fill those gaps.

In [52]:
# Calculate the count of -999.00 values in each column
count_999 = (Arlington_df == -999.00).sum()
print(count_999.sort_values(ascending=False)) # There are some undefined data in two columns

ALLSKY_KT            90
CLRSKY_SFC_SW_DWN    64
YEAR                  0
WS10M_MIN             0
WD50M                 0
                     ..
T2MDEW                0
T2M                   0
ALLSKY_SFC_SW_DWN     0
DY                    0
DATE                  0
Length: 21, dtype: int64


In [53]:
# Filter the DataFrame for rows where either column has NaN
Arlington_df.replace(-999.00, np.nan, inplace=True)
undefined_rows = Arlington_df[Arlington_df['ALLSKY_KT'].isna() | Arlington_df['CLRSKY_SFC_SW_DWN'].isna()]

# Print the rows with undefined values
display(undefined_rows)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
1096,2024,1,2,2.67,2.68,NaN,1.11,-1.18,-0.04,101.22,...,6.32,3.13,3.19,317.19,6.85,7.87,5.32,2.55,317.75,2024-01-02
1097,2024,1,3,2.59,2.71,NaN,1.69,-0.95,0.37,100.94,...,3.15,1.51,1.63,247.94,4.07,5.52,1.84,3.68,248.50,2024-01-03
1098,2024,1,4,1.15,NaN,NaN,1.36,-1.37,-0.00,101.03,...,8.30,2.20,6.10,314.31,7.81,10.59,3.61,6.98,315.31,2024-01-04
1099,2024,1,5,2.74,NaN,NaN,-1.45,-4.35,-2.90,101.77,...,3.85,1.62,2.23,270.25,4.72,7.02,1.89,5.13,270.69,2024-01-05
1100,2024,1,6,0.25,NaN,NaN,1.02,0.55,0.79,100.40,...,7.30,2.59,4.71,132.75,6.73,8.98,4.76,4.22,134.38,2024-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,1.03,NaN,NaN,7.55,6.65,7.10,100.97,...,3.89,1.27,2.62,131.56,3.55,5.34,2.41,2.93,132.56,2024-03-27
1182,2024,3,28,1.24,NaN,NaN,6.85,4.46,5.65,100.68,...,8.65,0.99,7.66,304.88,7.48,10.83,1.63,9.19,305.06,2024-03-28
1183,2024,3,29,5.88,NaN,NaN,7.04,2.35,4.69,100.47,...,9.83,3.72,6.11,308.12,9.54,12.87,5.89,6.98,308.50,2024-03-29
1184,2024,3,30,3.18,NaN,NaN,9.04,5.31,7.17,100.30,...,5.30,2.59,2.71,250.94,6.10,8.95,3.47,5.47,252.19,2024-03-30


**NOTE**: For the columns `ALLSKY_KT` and `CLRSKY_SFC_SW_DWN`, There exists undefined numbers。For this problem, we only find the situation in year 2024, so there may have some technical issue for the data collection. After checking other datasets from NASA POWER, we realize the same situation. Imputation for the entire missing data from 2024 is not suitable and makes the data not effective, so that we decide to remove the data from 2024 for more precise and accurate analysis. Then, all datasets from NASA POWER are from 2021.01.01 to 2023.12.31.


In [54]:
# Filter out the year 2024
filtered_df = Arlington_df[Arlington_df['YEAR'] != 2024]

# Verify the operation by checking unique years remaining in the dataset
print("Unique years remaining in the dataset:", filtered_df['YEAR'].unique())

Unique years remaining in the dataset: [2021 2022 2023]


In [55]:
# Re-check the missing data
pd.options.display.max_rows = 25
display(filtered_df.isna().sum()) # no missing value

YEAR                 0
MO                   0
DY                   0
ALLSKY_SFC_SW_DWN    0
CLRSKY_SFC_SW_DWN    0
ALLSKY_KT            0
T2M                  0
T2MDEW               0
T2MWET               0
PS                   0
WS10M                0
WS10M_MAX            0
WS10M_MIN            0
WS10M_RANGE          0
WD10M                0
WS50M                0
WS50M_MAX            0
WS50M_MIN            0
WS50M_RANGE          0
WD50M                0
DATE                 0
dtype: int64

In [56]:
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Cleaned/Arlington_POWER_clean.csv', index=False)

In [57]:
display(filtered_df) # now we have 1095 rows and 21 columns for ARLINGTON datasets

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.47,2.49,0.12,1.07,-0.09,0.49,101.79,...,5.23,1.53,3.71,89.69,5.63,7.25,2.84,4.41,91.75,2021-01-01
1,2021,1,2,2.20,2.76,0.54,4.35,2.97,3.66,100.68,...,6.74,2.90,3.84,223.38,6.85,9.28,5.45,3.84,223.81,2021-01-02
2,2021,1,3,0.47,2.65,0.11,1.99,1.27,1.63,100.69,...,4.94,2.70,2.24,175.56,5.96,8.43,3.60,4.83,176.69,2021-01-03
3,2021,1,4,1.31,2.53,0.31,2.22,0.28,1.25,100.74,...,4.76,1.48,3.28,335.19,4.65,8.35,2.23,6.12,335.62,2021-01-04
4,2021,1,5,1.03,2.53,0.25,2.36,0.71,1.54,100.56,...,4.09,1.13,2.95,329.69,3.49,7.49,1.26,6.23,329.69,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023,12,27,0.94,2.49,0.23,9.34,8.94,9.15,100.47,...,7.10,0.40,6.70,134.94,6.12,10.21,0.61,9.60,136.62,2023-12-27
1091,2023,12,28,1.71,2.55,0.42,9.94,8.43,9.19,99.80,...,4.66,2.21,2.45,254.00,5.52,7.14,4.16,2.98,254.62,2023-12-28
1092,2023,12,29,1.71,2.72,0.42,6.56,3.26,4.91,99.89,...,6.24,2.26,3.98,311.38,6.22,9.01,2.82,6.19,311.56,2023-12-29
1093,2023,12,30,1.61,2.78,0.40,3.69,0.97,2.33,100.10,...,7.12,3.64,3.48,270.25,7.78,8.93,6.52,2.41,270.81,2023-12-30


### Richmond

In [58]:
Richmond_SolarTemp = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Richmond_Daily_Temp_Solar.csv")
Richmond_SolarTemp.head()

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN
0,2021,1,1,0.68,2.62,0.16,312.37,13.16,49.02,1.96,0.03,0.14,4.00,3.04,3.51,4.00,2.27,5.03,2.76
1,2021,1,2,2.50,2.82,0.59,306.83,44.61,50.57,5.45,0.08,0.38,8.51,7.00,7.75,8.03,10.30,14.82,4.51
2,2021,1,3,0.85,2.82,0.20,320.11,16.28,52.02,2.29,0.03,0.16,5.80,5.29,5.55,5.72,5.45,7.76,2.31
3,2021,1,4,1.41,2.83,0.33,301.93,25.85,50.29,3.48,0.04,0.20,4.34,2.11,3.23,3.81,8.03,8.94,0.90
4,2021,1,5,1.77,2.82,0.41,293.25,31.47,49.09,4.07,0.05,0.22,2.53,1.23,1.88,1.83,8.34,7.63,-0.71


In [59]:
Richmond_Wind = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Richmond_Daily_Wind.csv")
Richmond_Wind.head()

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,WS2M,PS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M
0,2021,1,1,0.68,2.62,0.69,101.81,2.03,2.75,0.84,1.91,94.25,3.62,4.23,2.22,2.02,95.81
1,2021,1,2,2.50,2.82,0.91,100.83,2.54,3.55,0.92,2.63,206.62,4.40,6.12,1.59,4.52,193.06
2,2021,1,3,0.85,2.82,0.79,100.70,2.38,3.23,1.39,1.84,199.69,4.20,5.32,3.09,2.23,201.75
3,2021,1,4,1.41,2.83,0.66,100.90,1.70,2.54,0.99,1.55,306.88,2.77,4.48,1.37,3.11,308.50
4,2021,1,5,1.77,2.82,0.62,100.71,1.67,2.39,0.32,2.06,308.31,3.12,5.38,0.41,4.96,307.94


In [60]:
# Merging DataFrames
Richmond = pd.merge(Richmond_SolarTemp, Richmond_Wind, on=['YEAR', 'MO', 'DY'])

# Selecting the variables that are needed for modeling
columns_to_select = [
    'YEAR', 'MO', 'DY',
    'ALLSKY_SFC_SW_DWN_x', 'CLRSKY_SFC_SW_DWN_x',
    'ALLSKY_KT', 'T2M', 'T2MDEW', 'T2MWET', 'PS',
    'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WD10M',
    'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE', 'WD50M'
]

# Selecting the columns from the DataFrame
Richmond_df = Richmond[columns_to_select]

# Changing variables name
Richmond_df = Richmond_df.rename(columns={
    'ALLSKY_SFC_SW_DWN_x': 'ALLSKY_SFC_SW_DWN',
    'CLRSKY_SFC_SW_DWN_x': 'CLRSKY_SFC_SW_DWN'
})

# Creating a column named "Date" by transfering the time variables to date
Richmond_df['DATE'] = pd.to_datetime({'year': Richmond_df['YEAR'], 'month': Richmond_df['MO'], 'day': Richmond_df['DY']})

# Displaying the dataframe
pd.options.display.max_rows = 10 
display(Richmond_df)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.68,2.62,0.16,4.00,3.04,3.51,101.81,...,2.75,0.84,1.91,94.25,3.62,4.23,2.22,2.02,95.81,2021-01-01
1,2021,1,2,2.50,2.82,0.59,8.51,7.00,7.75,100.83,...,3.55,0.92,2.63,206.62,4.40,6.12,1.59,4.52,193.06,2021-01-02
2,2021,1,3,0.85,2.82,0.20,5.80,5.29,5.55,100.70,...,3.23,1.39,1.84,199.69,4.20,5.32,3.09,2.23,201.75,2021-01-03
3,2021,1,4,1.41,2.83,0.33,4.34,2.11,3.23,100.90,...,2.54,0.99,1.55,306.88,2.77,4.48,1.37,3.11,308.50,2021-01-04
4,2021,1,5,1.77,2.82,0.41,2.53,1.23,1.88,100.71,...,2.39,0.32,2.06,308.31,3.12,5.38,0.41,4.96,307.94,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,0.56,-999.00,-999.00,8.62,7.40,8.01,101.02,...,2.45,0.68,1.77,97.00,2.41,3.41,1.36,2.05,98.00,2024-03-27
1182,2024,3,28,1.01,-999.00,-999.00,7.96,6.28,7.12,100.77,...,5.21,1.09,4.12,274.81,5.42,7.78,2.64,5.14,261.00,2024-03-28
1183,2024,3,29,5.61,-999.00,-999.00,9.33,5.30,7.32,100.71,...,5.93,2.34,3.59,289.88,6.79,9.67,3.36,6.32,291.44,2024-03-29
1184,2024,3,30,5.40,-999.00,-999.00,12.77,9.18,10.98,100.52,...,3.78,1.07,2.71,246.00,4.93,8.38,1.37,7.02,247.69,2024-03-30


In [61]:
# checking the data types
print(Richmond_df.dtypes) # nothing needs change

YEAR                          int64
MO                            int64
DY                            int64
ALLSKY_SFC_SW_DWN           float64
CLRSKY_SFC_SW_DWN           float64
                          ...      
WS50M_MAX                   float64
WS50M_MIN                   float64
WS50M_RANGE                 float64
WD50M                       float64
DATE                 datetime64[ns]
Length: 21, dtype: object


In [62]:
# Calculate the count of -999.00 values in each column
count_999 = (Richmond_df == -999.00).sum()
print(count_999.sort_values(ascending=False)) # There are some undefined data in two columns

ALLSKY_KT            90
CLRSKY_SFC_SW_DWN    62
YEAR                  0
WS10M_MIN             0
WD50M                 0
                     ..
T2MDEW                0
T2M                   0
ALLSKY_SFC_SW_DWN     0
DY                    0
DATE                  0
Length: 21, dtype: int64


In [63]:
# Filter the DataFrame for rows where either column has NaN
Richmond_df.replace(-999.00, np.nan, inplace=True)
undefined_rows = Richmond_df[Richmond_df['ALLSKY_KT'].isna() | Richmond_df['CLRSKY_SFC_SW_DWN'].isna()]

# Print the rows with undefined values
display(undefined_rows)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
1096,2024,1,2,2.81,2.82,NaN,2.33,-1.18,0.58,101.37,...,3.82,1.49,2.32,315.75,4.48,6.45,2.10,4.35,316.75,2024-01-02
1097,2024,1,3,2.51,NaN,NaN,2.59,-0.54,1.03,101.08,...,2.11,0.59,1.52,219.31,2.40,3.97,0.62,3.34,219.38,2024-01-03
1098,2024,1,4,1.15,NaN,NaN,2.48,-0.50,0.99,101.15,...,5.27,1.66,3.61,325.19,5.80,7.91,2.97,4.95,326.06,2024-01-04
1099,2024,1,5,2.90,NaN,NaN,-0.95,-4.48,-2.72,101.92,...,2.52,0.23,2.30,195.69,3.70,5.78,0.29,5.49,211.06,2024-01-05
1100,2024,1,6,0.31,NaN,NaN,3.01,2.55,2.78,100.42,...,4.82,1.41,3.41,180.25,4.85,8.43,2.95,5.48,182.44,2024-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,0.56,NaN,NaN,8.62,7.40,8.01,101.02,...,2.45,0.68,1.77,97.00,2.41,3.41,1.36,2.05,98.00,2024-03-27
1182,2024,3,28,1.01,NaN,NaN,7.96,6.28,7.12,100.77,...,5.21,1.09,4.12,274.81,5.42,7.78,2.64,5.14,261.00,2024-03-28
1183,2024,3,29,5.61,NaN,NaN,9.33,5.30,7.32,100.71,...,5.93,2.34,3.59,289.88,6.79,9.67,3.36,6.32,291.44,2024-03-29
1184,2024,3,30,5.40,NaN,NaN,12.77,9.18,10.98,100.52,...,3.78,1.07,2.71,246.00,4.93,8.38,1.37,7.02,247.69,2024-03-30


In [64]:
# Filter out the year 2024
filtered_df = Richmond_df[Richmond_df['YEAR'] != 2024]

# Verify the operation by checking unique years remaining in the dataset
print("Unique years remaining in the dataset:", filtered_df['YEAR'].unique())

Unique years remaining in the dataset: [2021 2022 2023]


In [65]:
# Re-check the missing data
pd.options.display.max_rows = 25
display(filtered_df.isna().sum()) # no missing value

YEAR                 0
MO                   0
DY                   0
ALLSKY_SFC_SW_DWN    0
CLRSKY_SFC_SW_DWN    0
ALLSKY_KT            0
T2M                  0
T2MDEW               0
T2MWET               0
PS                   0
WS10M                0
WS10M_MAX            0
WS10M_MIN            0
WS10M_RANGE          0
WD10M                0
WS50M                0
WS50M_MAX            0
WS50M_MIN            0
WS50M_RANGE          0
WD50M                0
DATE                 0
dtype: int64

In [66]:
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Cleaned/Richmond_POWER_clean.csv', index=False)

In [67]:
display(filtered_df)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.68,2.62,0.16,4.00,3.04,3.51,101.81,...,2.75,0.84,1.91,94.25,3.62,4.23,2.22,2.02,95.81,2021-01-01
1,2021,1,2,2.50,2.82,0.59,8.51,7.00,7.75,100.83,...,3.55,0.92,2.63,206.62,4.40,6.12,1.59,4.52,193.06,2021-01-02
2,2021,1,3,0.85,2.82,0.20,5.80,5.29,5.55,100.70,...,3.23,1.39,1.84,199.69,4.20,5.32,3.09,2.23,201.75,2021-01-03
3,2021,1,4,1.41,2.83,0.33,4.34,2.11,3.23,100.90,...,2.54,0.99,1.55,306.88,2.77,4.48,1.37,3.11,308.50,2021-01-04
4,2021,1,5,1.77,2.82,0.41,2.53,1.23,1.88,100.71,...,2.39,0.32,2.06,308.31,3.12,5.38,0.41,4.96,307.94,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023,12,27,1.07,2.65,0.26,12.22,11.96,12.08,100.41,...,3.60,0.90,2.70,138.00,4.41,5.62,1.36,4.27,139.75,2023-12-27
1091,2023,12,28,1.20,2.79,0.28,10.12,9.27,9.69,99.99,...,2.35,1.21,1.14,266.88,3.92,5.90,1.61,4.29,268.12,2023-12-28
1092,2023,12,29,2.39,2.85,0.56,7.15,5.29,6.22,100.04,...,3.33,0.52,2.81,234.75,3.11,5.40,0.72,4.69,236.69,2023-12-29
1093,2023,12,30,2.53,2.95,0.59,3.65,1.26,2.46,100.43,...,4.35,2.11,2.24,252.75,5.29,6.65,4.27,2.38,254.25,2023-12-30


### Shenandoah

In [68]:
Shenandoah_SolarTemp = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Shenandoah_Daily_Solar_Temp.csv")
Shenandoah_SolarTemp.head()

,YEAR,MO,DY,T2M,T2MDEW,T2MWET,TS,T2M_RANGE,T2M_MAX,T2M_MIN,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,ALLSKY_SFC_LW_DWN,ALLSKY_SFC_PAR_TOT,CLRSKY_SFC_PAR_TOT,ALLSKY_SFC_UVA,ALLSKY_SFC_UVB,ALLSKY_SFC_UV_INDEX
0,2021,1,1,-0.70,-1.96,-1.33,-0.65,1.36,0.19,-1.17,0.45,2.42,0.11,298.48,8.57,44.27,1.29,0.02,0.09
1,2021,1,2,2.70,2.02,2.36,1.61,7.70,7.22,-0.49,2.10,2.81,0.51,287.91,37.98,50.38,4.88,0.07,0.34
2,2021,1,3,0.71,0.62,0.67,0.06,6.71,4.35,-2.37,0.68,2.53,0.16,314.30,12.95,44.64,1.89,0.02,0.12
3,2021,1,4,0.51,-1.43,-0.46,-0.82,7.41,4.54,-2.87,0.72,2.51,0.17,303.56,13.80,43.36,2.00,0.02,0.11
4,2021,1,5,-0.55,-2.17,-1.36,-1.74,6.64,3.46,-3.19,1.55,2.75,0.37,295.50,27.90,48.01,3.70,0.04,0.19


In [69]:
Shenandoah_Wind = pd.read_csv("/Users/crystal/Desktop/ANLY5550/Data/Raw/POWER_Shenandoah_Daily_Wind.csv")
Shenandoah_Wind.head()

,YEAR,MO,DY,PS,WS10M,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,WS2M
0,2021,1,1,96.65,2.23,3.00,1.50,1.50,126.00,3.52,4.79,2.10,2.69,127.12,0.45,2.42,1.05
1,2021,1,2,95.76,2.66,3.66,1.77,1.89,198.31,4.66,6.73,2.76,3.96,198.88,2.10,2.81,1.10
2,2021,1,3,95.70,1.74,2.77,0.75,2.02,210.31,3.40,6.13,1.25,4.88,213.88,0.68,2.53,0.62
3,2021,1,4,95.80,1.66,2.51,0.52,1.98,301.00,2.82,4.96,0.70,4.26,300.62,0.72,2.51,0.73
4,2021,1,5,95.62,1.70,2.59,0.66,1.94,317.31,2.95,5.48,0.87,4.62,316.19,1.55,2.75,0.72


In [70]:
# Merging DataFrames
Shenandoah = pd.merge(Shenandoah_SolarTemp, Shenandoah_Wind, on=['YEAR', 'MO', 'DY'])

# Selecting the variables that are needed for modeling
columns_to_select = [
    'YEAR', 'MO', 'DY',
    'ALLSKY_SFC_SW_DWN_x', 'CLRSKY_SFC_SW_DWN_x',
    'ALLSKY_KT', 'T2M', 'T2MDEW', 'T2MWET', 'PS',
    'WS10M', 'WS10M_MAX', 'WS10M_MIN', 'WS10M_RANGE', 'WD10M',
    'WS50M', 'WS50M_MAX', 'WS50M_MIN', 'WS50M_RANGE', 'WD50M'
]

# Selecting the columns from the DataFrame
Shenandoah_df = Shenandoah[columns_to_select]

# Changing variables name
Shenandoah_df = Shenandoah_df.rename(columns={
    'ALLSKY_SFC_SW_DWN_x': 'ALLSKY_SFC_SW_DWN',
    'CLRSKY_SFC_SW_DWN_x': 'CLRSKY_SFC_SW_DWN'
})

# Creating a column named "Date" by transfering the time variables to date
Shenandoah_df['DATE'] = pd.to_datetime({'year': Shenandoah_df['YEAR'], 'month': Shenandoah_df['MO'], 'day': Shenandoah_df['DY']})

# Displaying the dataframe
pd.options.display.max_rows = 10 
display(Shenandoah_df)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.45,2.42,0.11,-0.70,-1.96,-1.33,96.65,...,3.00,1.50,1.50,126.00,3.52,4.79,2.10,2.69,127.12,2021-01-01
1,2021,1,2,2.10,2.81,0.51,2.70,2.02,2.36,95.76,...,3.66,1.77,1.89,198.31,4.66,6.73,2.76,3.96,198.88,2021-01-02
2,2021,1,3,0.68,2.53,0.16,0.71,0.62,0.67,95.70,...,2.77,0.75,2.02,210.31,3.40,6.13,1.25,4.88,213.88,2021-01-03
3,2021,1,4,0.72,2.51,0.17,0.51,-1.43,-0.46,95.80,...,2.51,0.52,1.98,301.00,2.82,4.96,0.70,4.26,300.62,2021-01-04
4,2021,1,5,1.55,2.75,0.37,-0.55,-2.17,-1.36,95.62,...,2.59,0.66,1.94,317.31,2.95,5.48,0.87,4.62,316.19,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,1.82,-999.00,-999.00,7.97,6.91,7.44,95.93,...,3.04,0.55,2.48,154.81,2.95,5.45,0.84,4.60,156.38,2024-03-27
1182,2024,3,28,3.10,-999.00,-999.00,5.31,1.61,3.46,95.88,...,5.03,2.14,2.89,328.25,6.54,7.79,4.61,3.18,328.25,2024-03-28
1183,2024,3,29,5.68,-999.00,-999.00,6.49,2.01,4.25,95.68,...,7.68,2.77,4.91,301.00,8.09,11.07,4.61,6.46,301.19,2024-03-29
1184,2024,3,30,4.40,-999.00,-999.00,10.08,6.69,8.39,95.48,...,4.82,2.17,2.65,250.81,5.71,9.74,2.91,6.84,253.06,2024-03-30


In [71]:
# checking the data types
print(Shenandoah_df.dtypes) # nothing needs change

YEAR                          int64
MO                            int64
DY                            int64
ALLSKY_SFC_SW_DWN           float64
CLRSKY_SFC_SW_DWN           float64
                          ...      
WS50M_MAX                   float64
WS50M_MIN                   float64
WS50M_RANGE                 float64
WD50M                       float64
DATE                 datetime64[ns]
Length: 21, dtype: object


In [72]:
# Calculate the count of -999.00 values in each column
count_999 = (Shenandoah_df == -999.00).sum()
print(count_999.sort_values(ascending=False)) # There are some undefined data in two columns

# Filter the DataFrame for rows where either column has NaN
Shenandoah_df.replace(-999.00, np.nan, inplace=True)
undefined_rows = Shenandoah_df[Shenandoah_df['ALLSKY_KT'].isna() | Shenandoah_df['CLRSKY_SFC_SW_DWN'].isna()]

# Print the rows with undefined values
display(undefined_rows)

# Filter out the year 2024
filtered_df = Shenandoah_df[Shenandoah_df['YEAR'] != 2024]

# Verify the operation by checking unique years remaining in the dataset
print("Unique years remaining in the dataset:", filtered_df['YEAR'].unique())

ALLSKY_KT            90
CLRSKY_SFC_SW_DWN    64
YEAR                  0
WS10M_MIN             0
WD50M                 0
                     ..
T2MDEW                0
T2M                   0
ALLSKY_SFC_SW_DWN     0
DY                    0
DATE                  0
Length: 21, dtype: int64


,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
1096,2024,1,2,2.70,2.71,NaN,-0.97,-3.99,-2.48,96.26,...,4.44,1.69,2.74,311.00,4.84,6.46,2.71,3.75,310.75,2024-01-02
1097,2024,1,3,2.47,NaN,NaN,0.23,-3.35,-1.56,95.89,...,2.04,0.81,1.23,244.50,2.67,3.67,1.11,2.56,246.75,2024-01-03
1098,2024,1,4,1.16,NaN,NaN,-1.25,-4.55,-2.90,96.11,...,5.66,1.91,3.74,325.69,6.27,7.97,3.85,4.12,326.06,2024-01-04
1099,2024,1,5,2.81,NaN,NaN,-1.83,-5.24,-3.54,96.61,...,3.25,0.79,2.46,234.94,3.48,5.46,1.09,4.37,236.12,2024-01-05
1100,2024,1,6,0.19,NaN,NaN,-2.08,-2.97,-2.52,95.33,...,3.56,0.83,2.73,202.19,3.89,5.05,1.09,3.96,204.00,2024-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1181,2024,3,27,1.82,NaN,NaN,7.97,6.91,7.44,95.93,...,3.04,0.55,2.48,154.81,2.95,5.45,0.84,4.60,156.38,2024-03-27
1182,2024,3,28,3.10,NaN,NaN,5.31,1.61,3.46,95.88,...,5.03,2.14,2.89,328.25,6.54,7.79,4.61,3.18,328.25,2024-03-28
1183,2024,3,29,5.68,NaN,NaN,6.49,2.01,4.25,95.68,...,7.68,2.77,4.91,301.00,8.09,11.07,4.61,6.46,301.19,2024-03-29
1184,2024,3,30,4.40,NaN,NaN,10.08,6.69,8.39,95.48,...,4.82,2.17,2.65,250.81,5.71,9.74,2.91,6.84,253.06,2024-03-30


Unique years remaining in the dataset: [2021 2022 2023]


In [73]:
# Re-check the missing data
pd.options.display.max_rows = 25
display(filtered_df.isna().sum()) # no missing value

YEAR                 0
MO                   0
DY                   0
ALLSKY_SFC_SW_DWN    0
CLRSKY_SFC_SW_DWN    0
ALLSKY_KT            0
T2M                  0
T2MDEW               0
T2MWET               0
PS                   0
WS10M                0
WS10M_MAX            0
WS10M_MIN            0
WS10M_RANGE          0
WD10M                0
WS50M                0
WS50M_MAX            0
WS50M_MIN            0
WS50M_RANGE          0
WD50M                0
DATE                 0
dtype: int64

In [74]:
# Save the filtered DataFrame to a new CSV file
filtered_df.to_csv('/Users/crystal/Desktop/ANLY5550/Data/Cleaned/Shenandoah_POWER_clean.csv', index=False)

In [75]:
display(filtered_df)

,YEAR,MO,DY,ALLSKY_SFC_SW_DWN,CLRSKY_SFC_SW_DWN,ALLSKY_KT,T2M,T2MDEW,T2MWET,PS,...,WS10M_MAX,WS10M_MIN,WS10M_RANGE,WD10M,WS50M,WS50M_MAX,WS50M_MIN,WS50M_RANGE,WD50M,DATE
0,2021,1,1,0.45,2.42,0.11,-0.70,-1.96,-1.33,96.65,...,3.00,1.50,1.50,126.00,3.52,4.79,2.10,2.69,127.12,2021-01-01
1,2021,1,2,2.10,2.81,0.51,2.70,2.02,2.36,95.76,...,3.66,1.77,1.89,198.31,4.66,6.73,2.76,3.96,198.88,2021-01-02
2,2021,1,3,0.68,2.53,0.16,0.71,0.62,0.67,95.70,...,2.77,0.75,2.02,210.31,3.40,6.13,1.25,4.88,213.88,2021-01-03
3,2021,1,4,0.72,2.51,0.17,0.51,-1.43,-0.46,95.80,...,2.51,0.52,1.98,301.00,2.82,4.96,0.70,4.26,300.62,2021-01-04
4,2021,1,5,1.55,2.75,0.37,-0.55,-2.17,-1.36,95.62,...,2.59,0.66,1.94,317.31,2.95,5.48,0.87,4.62,316.19,2021-01-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,2023,12,27,0.91,2.55,0.23,7.86,7.75,7.80,95.49,...,3.13,0.27,2.86,187.38,3.57,5.20,0.52,4.67,188.00,2023-12-27
1091,2023,12,28,2.15,2.80,0.53,7.27,5.15,6.21,95.04,...,2.58,1.18,1.40,303.06,3.52,5.49,1.92,3.57,302.25,2023-12-28
1092,2023,12,29,1.55,2.75,0.38,2.29,-0.23,1.03,95.06,...,3.18,0.84,2.34,296.00,3.98,6.28,1.00,5.28,295.88,2023-12-29
1093,2023,12,30,1.68,2.81,0.41,0.48,-2.07,-0.80,95.27,...,4.74,2.36,2.39,276.06,5.65,6.58,4.89,1.68,276.88,2023-12-30


In [76]:
tracker.stop()

3.5557291926970064e-07